# Part 5: Parts of Speech Tagging

<hr>

In [45]:
# Import Custom Libraries
from materials.code import utils

# Import forign libraries
import importlib
import gensim
import nltk
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint
import pyarrow

# Download some datasets
!python3 -m spacy download en
!python3 -c  'import nltk;nltk.download("stopwords")'

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/ghamut/Documents/course-materials/homework/HW4/venv/lib/python3.7/site-packages/en_core_web_sm
-->
/Users/ghamut/Documents/course-materials/homework/HW4/venv/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1056)>


<br><br>
###  Introduction
Nouns, verbs, pronouns, prepositions, adverbs, conjunctions, participles, and articles - all of these are **parts of speech** (POS). Knowing the POS tag of a word might inform the way we interpret it, and its context. Consider the sentence:

> *I recieved a check*

Depending on how we interpret the POS class of the word `check`, the sentence above could take on a different meaning. If I was told that `check` is a `noun`, then I would interpret the sentence as referring to a payment; but if I was told that `check` was a `verb`, I would interpret this sentence as referring to an inspection of some kind. For this reason (among others), POS tagging is a topic of interest to the NLP community. 

### Tagging with SpaCy
[`SpaCy`](https://spacy.io/) is python NLP library with several useful tools, including a POS tagger. I strongly encourage you to experiment with `SpaCy` on your own, but for the purposes of the tutorial, I've written a simple wrapper that uses it to exact POS tags, and some other useful text features. You can find my code in `materials/code/pos.py`. 

Lets see an example:

In [47]:
# import the 
from materials.code import pos

#-------------------------------------------------------------------------
# Importing Text
#-------------------------------------------------------------------------
text            = "It costs $1,000.00 for a round trip to London! I am excited. Run, runing, ran"
pos_text        = pos.pos(text)
pos_text.process()
featurized_text = pos_text.features

In [17]:
featurized_text

,tokens,sentence_num,tags,children,dependencies,pos,lemma,is_stop,is_whitespace,is_symbol,...,is_numeric,is_units,is_currency,is_quotes,is_punct,is_hyphens,is_icons,prob,cluster,entity
0,It,0,PRP,[],nsubj,PRON,-PRON-,False,False,False,...,False,False,False,False,False,False,False,-20.0,0,
1,costs,0,VBZ,"[It, 1,000.00, for, !]",ROOT,VERB,cost,False,False,False,...,False,False,False,False,False,False,False,-20.0,0,
2,$,0,$,[],nmod,SYM,$,False,False,True,...,False,False,False,False,False,False,False,-20.0,0,
3,"1,000.00",0,CD,[$],dobj,NUM,"1,000.00",False,False,False,...,True,False,False,False,False,False,False,-20.0,0,MONEY
4,for,0,IN,[trip],prep,ADP,for,True,False,False,...,False,False,False,False,False,False,False,-20.0,0,
5,a,0,DT,[],det,DET,a,True,False,False,...,False,False,False,False,False,False,False,-20.0,0,
6,round,0,JJ,[],amod,ADJ,round,False,False,False,...,False,False,False,False,False,False,False,-20.0,0,
7,trip,0,NN,"[a, round, to]",pobj,NOUN,trip,False,False,False,...,False,False,False,False,False,False,False,-20.0,0,
8,to,0,IN,[London],prep,ADP,to,True,False,False,...,False,False,False,False,False,False,False,-20.0,0,
9,London,0,NNP,[],pobj,PROPN,London,False,False,False,...,False,False,False,False,False,False,False,-20.0,0,GPE


<br>The function I've written above tokenizes the sentences and provides, for each token, several features. I store the results in a [pandas](https://pandas.pydata.org/) dataframe so it's easier to visualize. If you've never used dataframes in python, and prefer to work with dictionaries, you can cast this representation back [to a dict](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_dict.html). The 6th column in the above table (`pos`) contains the parts-of-speech tags generated by `SpaCy` for each word in the text. You can find more information about what each of the POS tags mean [here](https://spacy.io/api/annotation), but I've also pasted some information from the SpaCy on-line documentation below:

* `ADJ`: adjective, e.g. big, old, green, incomprehensible, first
* `ADP`: adposition, e.g. in, to, during
* `ADV`: adverb, e.g. very, tomorrow, down, where, there
* `AUX`: auxiliary, e.g. is, has (done), will (do), should (do)
* `CONJ`: conjunction, e.g. and, or, but
* `CCONJ`: coordinating conjunction, e.g. and, or, but
* `DET`: determiner, e.g. a, an, the
* `INTJ`: interjection, e.g. psst, ouch, bravo, hello
* `NOUN`: noun, e.g. girl, cat, tree, air, beauty
* `NUM`: numeral, e.g. 1, 2017, one, seventy-seven, IV, MMXIV
* `PART`: particle, e.g. ’s, not,
* `PRON`: pronoun, e.g I, you, he, she, myself, themselves, somebody
* `PROPN`: proper noun, e.g. Mary, John, London, NATO, HBO
* `PUNCT`: punctuation, e.g. ., (, ), ?
* `SCONJ`: subordinating conjunction, e.g. if, while, that
* `SYM`: symbol, e.g. $, %, §, ©, +, −, ×, ÷, =, :), 😝
* `VERB`: verb, e.g. run, runs, running, eat, ate, eating
* `X`: other, e.g. sfpksdpsxmsa
* `SPACE`: space, e.g.


<hr> 

## Learning Exercise 5: 
#### Worth 1/5 Points
#### A. Word2Vec with POS 
Update the following CBOW `word2vec` implementation to include a one-hot coded vector that encodes the `POS` of the target word. Train the implementation with and without your modification. How do you expect the characteristics of the embeddings to change when adding when adding this information to the model?

In [27]:
#----------------------------------------------------------------
# Author: Srijith Rajamohan based off the work by Robert Guthrie
# https://srijithr.gitlab.io/post/word2vec/
#----------------------------------------------------------------

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import urllib.request
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

torch.manual_seed(1)

CONTEXT_SIZE  = 3
EMBEDDING_DIM = 10

test_sentence = """Empathy for the poor may not come easily to people who never experienced it. They may blame the victims and insist their predicament can be overcome through determination and hard work.
But they may not realize that extreme poverty can be psychologically and physically incapacitating — a perpetual cycle of bad diets, health care and education exacerbated by the shaming and self-fulfilling prophecies that define it in the public imagination.
Gordon Parks — perhaps more than any artist — saw poverty as “the most savage of all human afflictions” and realized the power of empathy to help us understand it. It was neither an abstract problem nor political symbol, but something he endured growing up destitute in rural Kansas and having spent years documenting poverty throughout the world, including the United States.
That sensitivity informed “Freedom’s Fearful Foe: Poverty,” his celebrated photo essay published in Life magazine in June 1961. He took readers into the lives of a Brazilian boy, Flavio da Silva, and his family, who lived in the ramshackle Catacumba favela in the hills outside Rio de Janeiro. These stark photographs are the subject of a new book, “Gordon Parks: The Flavio Story” (Steidl/The Gordon Parks Foundation), which accompanies a traveling exhibition co-organized by the Ryerson Image Centre in Toronto, where it opens this week, and the J. Paul Getty Museum. Edited with texts by the exhibition’s co-curators, Paul Roth and Amanda Maddox, the book also includes a recent interview with Mr. da Silva and essays by Beatriz Jaguaribe, Maria Alice Rezende de Carvalho and Sérgio Burgi.
""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)

def get_key(word_id):
    for key,val in word_to_ix.items():
        if(val == word_id):
            print(key)

def cluster_embeddings(filename,nclusters):
    X = np.load(filename)
    kmeans = KMeans(n_clusters=nclusters, random_state=0).fit(X)
    center = kmeans.cluster_centers_
    distances = euclidean_distances(X,center)

    for i in np.arange(0,distances.shape[1]):
        word_id = np.argmin(distances[:,i])
        print(word_id)
        get_key(word_id)

def read_data(file_path):
    tokenizer = RegexpTokenizer(r'\w+')
    data = urllib.request.urlopen(file_path)
    data = data.read().decode('utf8')
    tokenized_data = word_tokenize(data)
    stop_words = set(stopwords.words('english'))
    stop_words.update(['.',',',':',';','(',')','#','--','...','"'])
    cleaned_words = [ i for i in tokenized_data if i not in stop_words ]
    return(cleaned_words)

#test_sentence = read_data('https://www.gutenberg.org/files/57884/57884-0.txt')

ngrams = []
for i in range(len(test_sentence) - CONTEXT_SIZE):
    tup = [test_sentence[j] for j in np.arange(i , i + CONTEXT_SIZE) ]
    ngrams.append((tup,test_sentence[i + CONTEXT_SIZE]))
# print the first 3, just so you can see what they look like
#print(ngrams)


vocab = set(test_sentence)
print("Length of vocabulary",len(vocab))
word_to_ix = {word: i for i, word in enumerate(vocab)}


class CBOWModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOWModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))  # -1 implies size inferred for that index from the size of the data
        #print(np.mean(np.mean(self.linear2.weight.data.numpy())))
        out1 = F.relu(self.linear1(embeds)) # output of first layer
        out2 = self.linear2(out1)           # output of second layer
        #print(embeds)
        log_probs = F.log_softmax(out2, dim=1)
        return log_probs

    def predict(self,input):
        context_idxs = torch.tensor([word_to_ix[w] for w in input], dtype=torch.long)
        res = self.forward(context_idxs)
        res_arg = torch.argmax(res)
        res_val, res_ind = res.sort(descending=True)
        res_val = res_val[0][:3]
        res_ind = res_ind[0][:3]
        #print(res_val)
        #print(res_ind)
        for arg in zip(res_val,res_ind):
            #print(arg)
            print([(key,val,arg[0]) for key,val in word_to_ix.items() if val == arg[1]])

    def freeze_layer(self,layer):
        for name,child in model.named_children():
            print(name,child)
            if(name == layer):
                for names,params in child.named_parameters():
                    print(names,params)
                    print(params.size())
                    params.requires_grad= False

    def print_layer_parameters(self):
        for name,child in model.named_children():
                print(name,child)
                for names,params in child.named_parameters():
                    print(names,params)
                    print(params.size())

    def write_embedding_to_file(self,filename):
        for i in self.embeddings.parameters():
            weights = i.data.numpy()
        np.save(filename,weights)



losses        = []
loss_function = nn.NLLLoss()
model         = CBOWModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer     = optim.SGD(model.parameters(), lr=0.001)

# Freeze embedding layer
#model.freeze_layer('embeddings')


for epoch in range(400):
    total_loss = 0
    #------- Embedding layers are trained as well here ----#
    #lookup_tensor = torch.tensor([word_to_ix["poor"]], dtype=torch.long)
    #hello_embed = model.embeddings(lookup_tensor)
    #print(hello_embed)
    # -----------------------------------------------------#

    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        #print("Context id",context_idxs)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        #print(log_probs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        #print(loss)

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    #print(total_loss)
    losses.append(total_loss)
#print(losses)  # The loss decreased every iteration over the training data!

#Print the model layer parameters
#model.print_layer_parameters()

#Predict the next word given n context words
model.predict(['of','all','human'])
model.write_embedding_to_file('embeddings.npy')
cluster_embeddings('embeddings.npy',2)


Length of vocabulary 195
[('afflictions”', 2, tensor(-0.0359, grad_fn=<SelectBackward>))]
[('help', 92, tensor(-5.3056, grad_fn=<SelectBackward>))]
[('de', 136, tensor(-5.6212, grad_fn=<SelectBackward>))]
193
Silva,
74
rural


<span style="color:red"> INSERT AN INTERPRETATION OF YOUR RESULTS HERE </span>

<hr>
<h1><span style="color:red"> Self Assessment </span></h1>
Please provide an assessment of how successfully you accomplished the learning exercises in this assignment according to the instruction provided; do not assign yourself points for effort. This self assessment will be used as a starting point when I grade your assignments. Please note that if you over-estimate your grade on a given learning exercise, you will face a 50% penalty on the total points granted for that exercise. If you underestimate your grade, there will be no penalty.

* Learning Exercise: 
    * <span style="color:red">X</span>/1 points